
### **Atividade Prática: Introdução ao Processamento de Linguagem Natural (NLP)**  

#### **Objetivos da Atividade**  
1. Compreender os conceitos fundamentais do Processamento de Linguagem Natural (NLP) em português.  
2. Aplicar técnicas de pré-processamento de texto, incluindo remoção de stopwords, stemming e lematização.  
3. Explorar e comparar diferentes representações de texto (Bag of Words e TF-IDF).  
4. Desenvolver modelos de classificação de sentimentos e comparar métricas de desempenho.  

#### **Sumário**  
1. Introdução ao NLP.  
2. Importação das bibliotecas necessárias.  
3. Baixar os datasets de NLP.  
4. Análise do dataset e descrição.  
5. Expressões regulares para limpeza de texto.  
6. Remoção de stopwords.  
7. Lemmatização e Stemming (com tabela comparativa).  
8. Representação de texto com Bag of Words.  
9. Representação de texto com TF-IDF.  
10. Classificação de sentimentos.  
  

---

### **1) Introdução ao NLP**  
O Processamento de Linguagem Natural (NLP) é uma subárea da Inteligência Artificial que busca criar sistemas capazes de entender, interpretar e gerar texto ou fala em linguagem humana.  

Aplicações comuns de NLP incluem:  
- Análise de sentimentos.  
- Tradução automática.  
- Resumo de texto.  
- Sistemas de busca.  
- Assistentes virtuais.  




### **2) Importação das bibliotecas necessárias**  


In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
import pandas as pd
import random
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')  # Stemmer em português
nltk.download('punkt_tab')




### **3) Dataset sintético para classificação de sentimentos**  
Utilizaremos um dataset sintético em português com dados de sentimentos. Este consiste em reviews de produtos, onde cada texto é rotulado com um sentimento: positivo ou negativo. Esta tarefa é uma **classificação supervisionada**, pois usamos os rótulos (variável dependente) para treinar e avaliar o modelo.

**Descrição das variáveis**:  
- `review`: texto contendo a opinião do usuário.  
- `sentiment`: rótulo associado à opinião (positivo = 1, negativo = 0).  



In [164]:
# Expansão das listas de frases positivas e negativas
frases_positivas_treino = [
    "Adorei o produto! #satisfeito", "Excelente qualidade, recomendo! @lojaTop",
    "Superou minhas expectativas! Veja em https://produto.com/top",
    "Muito bom, chegou antes do prazo. #ótimaCompra",
    "Gostei bastante, produto perfeito. Obrigado @vendedor!",
    "Ótima experiência de compra. #Recomendo",
    "Funciona perfeitamente, estou muito satisfeito. Veja mais: https://review.com/produto",
    "Produto incrível, muito útil! @superLoja",
    "Chegou em perfeito estado e adorei. #Top",
    "Com certeza compraria novamente! Veja aqui: https://loja.com/perfeito",
    "Entrega rápida e eficiente. #rapidez",
    "Valeu muito a pena, recomendo! Confira @lojaBoa",
    "Produto cumpre o prometido. Veja em https://produto.com",
    "A embalagem estava impecável, nota 10. #embalagemTop",
    "Compra maravilhosa, super satisfeita! #Maravilhoso",
    "Atendimento incrível, me ajudaram muito. @AtendimentoPro",
    "Sem dúvidas, a melhor compra do ano! Veja: https://compras.com/melhor",
    "Estou impressionado com a qualidade. #QualidadeImpressionante",
    "Chegou antes do prazo e é tudo que eu precisava. #EntregaPerfeita",
    "Melhor custo-benefício, recomendo a todos! #custoBeneficio",
    "Incrível como o produto é eficiente. Confira: https://produto.com/eficiencia",
    "A equipe está de parabéns pelo serviço. @EquipeTop",
    "Produto fantástico, mudou minha rotina. Veja mais: https://produto.com/fantastico",
    "Superou todas as expectativas, parabéns @LojaBoa! #Top",
    "Recomendo para amigos e familiares, é excelente! #Recomendo"
]

frases_positivas_teste = [
    "A loja é excelente, tudo perfeito! #topLoja",
    "Adorei a embalagem, muito cuidadosa. @embalagemPro",
    "Recomendo sem dúvidas, ótimo serviço. Veja: https://servico.com/otimo",
    "Tudo veio conforme descrito, muito bom. #Confiável",
    "Ficou perfeito para o que eu precisava. #Perfeito",
    "O presente foi um sucesso! Obrigado @lojaPresente",
    "Muito melhor do que eu esperava. Confira mais: https://produto.com/incrivel",
    "Parabéns pelo ótimo atendimento! #obrigado",
    "Estou muito feliz com a compra! #feliz",
    "Material de ótima qualidade. #qualidade",
    "Fiquei impressionado com a qualidade. @marcaTop",
    "Compra totalmente satisfatória. Confira: https://compra.com/satisfeito",
    "Chegou rápido e o atendimento foi excelente. #RapidezEficiente",
    "Melhor loja que já comprei, tudo perfeito. @LojaTop",
    "O produto é tão bom que comprei mais um! #ProdutoTop",
    "Recomendo para qualquer pessoa que procure qualidade. @QualidadeBoa",
    "Vale cada centavo, produto de primeira. #Recomendo",
    "Ótima solução para minhas necessidades. Veja: https://produto.com/solucao",
    "Me surpreendeu em todos os aspectos. @LojaIncrivel",
    "Não esperava tanto, mas foi maravilhoso. #MaravilhosoExtra"
]

frases_negativas_treino = [
    "Produto horrível, não recomendo. #decepção", "Péssima experiência, muito ruim. @lojaRuim",
    "Chegou quebrado, total decepção. Veja: https://produto.com/quebrado",
    "Material de qualidade muito baixa. #ruim",
    "Não funciona como descrito, muito insatisfeito. Veja: https://review.com/insatisfeito",
    "Entrega atrasada, serviço ruim. #atraso",
    "Produto caro e de má qualidade. Veja @lojaCara",
    "Não gostei, nunca mais compro aqui. #péssimo",
    "Descrição enganosa, perdi dinheiro. Veja mais: https://enganoso.com",
    "Horrível, arrependido da compra. @lojaPéssima",
    "Experiência frustrante, não recomendo. #frustração",
    "Produto veio errado, muito descaso. Veja @suporteRuim",
    "Produto não entregou o que prometia, fiquei insatisfeito. #promessaFalsa",
    "Paguei caro e recebi algo muito inferior. Veja mais: @DescontoRuim",
    "Reclamei com o suporte e não me ajudaram. #SuportePéssimo",
    "É uma fraude, tomem cuidado. Veja: https://fraude.com",
    "Muito abaixo das expectativas. @ExpectativaRuim",
    "Entrega demorou semanas, total descaso. #AtrasoEntrega",
    "O produto veio com defeito e não funciona. #DefeitoProduto",
    "Não recomendo, foi uma experiência frustrante. Veja: https://frustrante.com"
]

frases_negativa_teste = [
    "Foi uma completa perda de tempo. Veja mais: https://perda.com",
    "O suporte da loja é péssimo. #suporteRuim",
    "Infelizmente não funcionou nada. Veja: https://produto.com/nãoFunciona",
    "Muito frágil, quebrou no primeiro uso. @produtoRuim",
    "Chegou sujo e mal embalado. #péssimo",
    "A descrição era completamente falsa. @falsidade",
    "Muito caro pelo que oferece. Veja: https://custo.com/alto",
    "Não atendeu as minhas expectativas. @lojaDecepcionante",
    "Infelizmente foi uma péssima compra. Veja: https://péssimo.com",
    "Não vale o preço pago. #caro",
    "Não comprem, é uma armadilha. Veja: https://cuidado.com",
    "Decepcionante, esperava mais. @compraRuim",
    "A qualidade é extremamente ruim. #ruim",
    "Produto com prazo de validade vencido. @ProblemaProduto",
    "Recebi algo completamente diferente do anunciado. Veja mais: https://enganoso.com",
    "O produto parou de funcionar após um dia de uso. #DurabilidadeRuim",
    "Muito caro e a qualidade não corresponde. @CustoBeneficioRuim",
    "Nunca mais compro dessa loja, foi decepcionante. #LojaRuim",
    "As especificações são totalmente falsas. Veja: https://produto.com/mentira",
    "Me arrependi completamente da compra. @Arrependimento"
]



### **Geraçaõ do conjunto de Treino**  

In [ ]:

# Função para gerar combinações de frases
def gerar_review(frases, num_frases=4):
    return " ".join(random.sample(frases, num_frases))

# Gerar exemplos sintéticos
reviews_train = []
sentiments_train = []

# Gerar 250 exemplos positivos
for _ in range(200):
    reviews_train.append(gerar_review(frases_positivas_treino, random.randint(1, 4)))  # Combina 1 a 5 frases positivas
    sentiments_train.append(1)

# Gerar 250 exemplos negativos
for _ in range(200):
    reviews_train.append(gerar_review(frases_negativas_treino, random.randint(1, 4)))  # Combina 1 a 5 frases negativas
    sentiments_train.append(0)

# Misturar os exemplos
data_treino = pd.DataFrame({'review': reviews_train, 'sentiment': sentiments_train}).sample(frac=1).reset_index(drop=True)

# Exibir as primeiras linhas do dataset
data_treino


### **Geraçaõ do conjunto de Teste**  

In [ ]:
# Gerar exemplos sintéticos
reviews_test = []
sentiments_test = []

# Gerar 250 exemplos positivos
for _ in range(150):
    reviews_test.append(gerar_review(frases_positivas_teste, random.randint(1, 4)))  # Combina 1 a 5 frases positivas
    sentiments_test.append(1)

# Gerar 250 exemplos negativos
for _ in range(150):
    reviews_test.append(gerar_review(frases_negativa_teste, random.randint(1, 4)))  # Combina 1 a 5 frases negativas
    sentiments_test.append(0)

# Misturar os exemplos
data_test = pd.DataFrame({'review': reviews_test, 'sentiment': sentiments_test}).sample(frac=1).reset_index(drop=True)

# Exibir as primeiras linhas do dataset
data_test

In [ ]:
# Análise de distribuição
print("\nDistribuição de classes:")
data_treino['sentiment'].value_counts()



In [ ]:
# Análise de distribuição
print("\nDistribuição de classes:")
data_test['sentiment'].value_counts()

### **5) Expressões Regulares para Limpeza de Texto**  

As expressões regulares (Regex) são ferramentas poderosas para identificar padrões em textos. Durante o pré-processamento, são usadas para:  
1. Remover links, hashtags e menções.  
2. Substituir caracteres especiais e números.  
3. Uniformizar o texto (ex.: converter para minúsculas).

**Fórmulas e exemplos**:  
- Para remover URLs: `re.sub(r"http\S+", "", texto)`  
- Para remover números: `re.sub(r"\d+", "", texto)`  



| **Expressão Regular** | **Descrição** | **Exemplo** |
|-----------------------|---------------|-------------|
| `\d`                  | Qualquer dígito (0-9) | `'123'.match(r'\d')` (True) |
| `\D`                  | Qualquer caractere que não seja dígito | `'a'.match(r'\D')` (True) |
| `\w`                  | Qualquer caractere alfanumérico (letras, números e _) | `'abc123'.match(r'\w')` (True) |
| `\W`                  | Qualquer caractere que não seja alfanumérico | `'@'.match(r'\W')` (True) |
| `\s`                  | Espaço em branco, incluindo espaços, tabs e novas linhas | `' '.match(r'\s')` (True) |
| `\S`                  | Qualquer caractere que não seja espaço em branco | `'a'.match(r'\S')` (True) |
| `^`                   | Início da string | `re.match(r'^a', 'abc')` (True) |
| `$`                   | Fim da string | `re.match(r'abc$', 'abc')` (True) |
| `.`                   | Qualquer caractere, exceto nova linha | `'a'.match(r'.')` (True) |
| `*`                   | Zero ou mais repetições do padrão anterior | `'aaaa'.match(r'a*')` (True) |
| `+`                   | Uma ou mais repetições do padrão anterior | `'aaa'.match(r'a+')` (True) |
| `?`                   | Zero ou uma repetição do padrão anterior | `'a'.match(r'a?')` (True) |
| `{n}`                 | Exatamente n repetições do padrão anterior | `'aaa'.match(r'a{3}')` (True) |
| `{n,}`                | Pelo menos n repetições do padrão anterior | `'aaaa'.match(r'a{3,}')` (True) |
| `{n,m}`               | Entre n e m repetições do padrão anterior | `'aabb'.match(r'a{2,3}')` (True) |
| `[]`                  | Conjunto de caracteres | `'a'.match(r'[abc]')` (True) |
| `|`                   | Ou | `'a'.match(r'a|b')` (True) |
| `()`                  | Agrupamento de padrões | `re.match(r'(abc)+', 'abcabc')` (True) |
| `\b`                  | Limite de palavra | `'abc'.match(r'\babc\b')` (True) |
| `\B`                  | Não-limite de palavra | `'abc'.match(r'\Babc\B')` (False) |



In [ ]:

# Função de limpeza
def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"@\w+", "", text)    # Remove menções
    text = re.sub(r"#\w+", "", text)    # Remove hashtags
    text = re.sub(r"[^a-zA-Záéíóúçãõ ]", " ", text)  # Remove caracteres especiais
    text = text.lower()  # Converte para minúsculas
    return text

data_treino['cleaned_review'] = data_treino['review'].apply(clean_text)
data_treino[['review', 'cleaned_review']]


In [ ]:
data_test['cleaned_review'] = data_test['review'].apply(clean_text)
data_test[['review', 'cleaned_review']]


### **6) Remoção de Stopwords**  

Stopwords são palavras frequentes (como "de", "a", "e") que normalmente não carregam informações úteis para o modelo.  

A remoção de stopwords reduz a dimensionalidade do texto e ajuda a focar em palavras mais relevantes.  


---

In [ ]:

# Stopwords em português
stop_words = set(stopwords.words('portuguese'))

# Remover "não" da lista de stopwords
stop_words.discard("não")

def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    return " ".join([word for word in words if word not in stop_words])

data_treino['no_stopwords'] = data_treino['cleaned_review'].apply(remove_stopwords)
data_treino[['cleaned_review', 'no_stopwords']]



In [ ]:
data_test['no_stopwords'] = data_test['cleaned_review'].apply(remove_stopwords)
data_test[['cleaned_review', 'no_stopwords']]

### **7) Lemmatização e Stemming**  

- **Stemming**: Reduz as palavras à sua raiz, sem considerar o significado. (Ex.: "correr", "correu" → "corr").  
- **Lemmatização**: Reduz as palavras à forma base considerando seu contexto gramatical. (Ex.: "correu" → "correr").  


In [ ]:
nlp = spacy.load('pt_core_news_sm')
# Função de Stemming
def stemming(text):
    stemmer = RSLPStemmer()
    words = word_tokenize(text, language='portuguese')
    return " ".join([stemmer.stem(word) for word in words])

# Função de Lemmatization
def lemmatization(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

data_treino['Stemmed'] = data_treino['no_stopwords'].apply(stemming)
data_treino['Lemmatized'] = data_treino['no_stopwords'].apply(lemmatization)

data_treino[['no_stopwords','Lemmatized','Stemmed']].head()


In [ ]:
data_test['Stemmed'] = data_test['no_stopwords'].apply(stemming)
data_test['Lemmatized'] = data_test['no_stopwords'].apply(lemmatization)

data_test[['no_stopwords','Lemmatized','Stemmed']].head()



### **8) Representação de Texto com Bag of Words**  
**Fundamentação Teórica**:  
O modelo **Bag of Words (BoW)** cria uma matriz de frequências de palavras.  

Seja um conjunto $( D )$ com $( n )$ documentos e $( m )$ termos:  
- Elemento $( A[i][j] )$ indica a frequência do termo $( j )$ no documento $( i)$.  



In [ ]:

cv = CountVectorizer()
X_cv = cv.fit_transform(data_treino['no_stopwords'])

print("Shape da matriz:", X_cv.shape)
print("Exemplo de matriz:",)
X_cv.toarray()[:5]


### **9) Representação de Texto com TF-IDF**  

A técnica **TF-IDF** atribui peso maior a termos relevantes e menos frequentes em um corpus.  

Equação:  
$
TF\text{-}IDF(t,d) = TF(t,d) \times \log\left(\frac{N}{DF(t)}\right)
$
Onde:  
- $( TF(t,d) )$: frequência do termo \( t \) no documento \( d \).  
- $( DF(t) $): número de documentos contendo o termo \( t \).  
- $( N $): total de documentos.  



In [ ]:

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(data_treino['no_stopwords'])

print("Shape da matriz:", X_tfidf.shape)
print("Exemplo de matriz:")
X_tfidf.toarray()[:5]



### **10) Classificação de Sentimentos**  

Uma tarefa importante em Processamento de Linguagem Natural (NLP) que visa identificar a atitude emocional expressa em um texto. Em geral, a classificação de sentimentos envolve a categorização de opiniões ou reviews em rótulos como **positivo**, **negativo** ou **neutro**. No contexto desta atividade, o objetivo é classificar reviews de produtos como **positivos** ou **negativos**.

A regressão logística foi escolhida como modelo para essa tarefa devido à sua simplicidade, eficiência e capacidade de lidar bem com tarefas de classificação binária, como esta.

##### **Desafios da Classificação de Sentimentos com Regressão Logística**
Embora a regressão logística seja uma boa escolha para tarefas de classificação binária, ela pode não capturar completamente as dependências mais complexas no texto, como as interações entre palavras em frases mais longas ou o contexto em que as palavras são usadas. Isso pode ser uma limitação quando se lida com textos mais complicados ou quando a relação entre as características e o sentimento não é linear.

##### **Avaliação do Modelo**
Para avaliar a performance do modelo de regressão logística, é fundamental utilizar métricas adequadas. Como o problema é de classificação binária, as principais métricas incluem:
- **Acurácia:** A proporção de classificações corretas em relação ao total de classificações.
- **Precisão e Recall:** Importantes quando se busca minimizar erros de classificação em uma das classes, como por exemplo, evitar a classificação incorreta de reviews positivas como negativas.
- **F1-Score:** A média harmônica entre precisão e recall, útil para avaliar o equilíbrio entre esses dois aspectos.
- **Matriz confusão:** permite indenticar a exata quantidade de Falsos Positivos, Falsos negativos e Verdadeios positivos.



In [ ]:
# Separar features (X) e rótulos (y)
X_train = data_treino['review']
y_train = data_treino['sentiment']

X_test = data_test['review']
y_test = data_test['sentiment']

# Vetorização usando CountVectorizer
vectorizer = CountVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Modelo de classificação
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Previsões
y_pred = model.predict(X_test_tfidf)

# Relatório de classificação
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão")
plt.show()


#### **10.1) Classificação de Sentimentos com pré-processamento Stemming**

In [ ]:
# Separar features (X) e rótulos (y)
X_train = data_treino['Stemmed']
y_train = data_treino['sentiment']

X_test = data_test['Stemmed']
y_test = data_test['sentiment']

# Vetorização usando CountVectorizer
vectorizer = CountVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Modelo de classificação
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Previsões
y_pred = model.predict(X_test_tfidf)

# Relatório de classificação
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão")
plt.show()


#### **10.2) Classificação de Sentimentos com pré-processamento e TF-IDF**

In [ ]:

# Vetorização usando TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Modelo de classificação
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Previsões
y_pred = model.predict(X_test_tfidf)

# Relatório de classificação
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão")
plt.show()


#### **10.3) Classificação de Sentimentos com pré-processamento, TF-IDF e utilizando Naive Bayes como modelo para classificação**


O **Naive Bayes** é um modelo probabilístico amplamente utilizado em tarefas de **classificação de texto**, incluindo a **classificação de sentimentos**. Seu principal fundamento é o **Teorema de Bayes**, que aplica a probabilidade condicional para prever a classe de uma instância com base nas suas características.

Neste caso, o objetivo é classificar **reviews de produtos** como **positivas** ou **negativas** com base nas palavras ou termos que aparecem no texto. O Naive Bayes é uma excelente escolha devido à sua simplicidade, eficiência e boa performance em problemas de texto.

###### **Princípio Básico do Naive Bayes**
O modelo de Naive Bayes baseia-se no **Teorema de Bayes**, que descreve a probabilidade de uma classe \( C \) dada uma instância de dados \( X \), que é composta por características (neste caso, as palavras do texto):

$
P(C|X) = \frac{P(X|C)P(C)}{P(X)}
$

Onde:
- $( P(C|X) )$ é a probabilidade de \( X \) pertencer à classe \( C \) (por exemplo, positivo ou negativo) dado \( X \) (as palavras no texto).
- $( P(X|C) )$ é a probabilidade de observar o conjunto de características \( X \) dado que a classe é \( C \).
- $( P(C) )$ é a probabilidade a priori de cada classe (por exemplo, a probabilidade de um review ser positivo ou negativo antes de observar o texto).
- $( P(X) )$ é a probabilidade de observar as características \( X \) em qualquer classe (também chamada de **evidência**).


No caso da **classificação de sentimentos**, o objetivo é determinar se um review é **positivo** ou **negativo** com base nas palavras presentes nele. Cada review pode ser visto como uma sequência de características (as palavras ou termos), e o Naive Bayes tenta calcular qual é a classe mais provável (positiva ou negativa) dado o conjunto de palavras.

O modelo aprende a **probabilidade a priori** de cada classe (quanto um review tende a ser positivo ou negativo) e a **probabilidade condicional** de cada palavra ocorrer em reviews positivos ou negativos. Com isso, ele é capaz de calcular a probabilidade de cada classe e escolher a classe com maior probabilidade.

###### **Vantagens do Naive Bayes**
- **Simplicidade:** O Naive Bayes é fácil de entender e implementar, o que o torna uma excelente escolha para tarefas iniciais de classificação de sentimentos.
- **Eficiência:** Devido à sua suposição de independência entre palavras, o modelo pode ser treinado rapidamente, mesmo em grandes volumes de dados de texto.
- **Bom Desempenho em Texto:** O Naive Bayes tem mostrado bons resultados em várias tarefas de NLP, como análise de sentimentos, detecção de spam, e outras, mesmo com a suposição de independência, que é muitas vezes não completamente verdadeira.
- **Robustez:** Mesmo quando a suposição de independência entre palavras não é totalmente válida, o modelo ainda pode funcionar bem, especialmente quando as palavras são indicativas de uma classe específica.

##### **Desvantagens**
- **Independência entre Características:** A principal limitação do Naive Bayes é a suposição de que as características (palavras) são independentes. Em textos, as palavras frequentemente dependem umas das outras, o que pode afetar a performance do modelo em textos mais complexos ou com dependências entre palavras.
- **Modelagem de Dependências Limitada:** Como o modelo não consegue capturar dependências entre palavras, ele pode falhar em capturar nuances de significado que dependem do contexto da palavra, como no caso de ironia ou sarcasmo.



In [ ]:
# Vetorização usando TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Modelo de classificação
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Previsões
y_pred = model.predict(X_test_tfidf)

# Relatório de classificação
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão")
plt.show()

#### **11) Atividade Opcioanal**


Para aprimorar o desempenho do modelo de classificação de sentimentos, explore diferentes combinações de técnicas de pré-processamento de texto, métodos de vetorização e algoritmos de classificação.

Considere as seguintes abordagens:

1. **Pré-processamento:**
   - Aplicação de **Lematização** em vez de Stemming.
   - Remoção de **stopwords** específicas do domínio.
.

2. **Vetorização:**
   - Utilização de **TF-IDF** com diferentes parâmetros.
  

3. **Algoritmos de Classificação:**
   - Aplicação de **Máquinas de Vetores de Suporte (SVM)**.
   - Teste com **Árvores de Decisão**.
   - Avaliação de **Redes Neurais Artificiais**.

Para cada combinação, avalie o desempenho utilizando métricas como acurácia, precisão, recall e F1-score.



In [ ]:
#Resposta